## Regional Ocean: Checking Open Boundary Conditions

In [ ]:
%load_ext autoreload
%autoreload 2

import xarray as xr
import numpy as np
import os
from pathlib import Path
import glob
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs

import regional_utils as utils

In [ ]:
case_name = ""  # "/glade/campaign/cgd/oce/projects/CROCODILE/workshops/2025/Diagnostics/CESM_Output/"
CESM_output_dir = ""  # "CROCODILE_tutorial_nwa12_MARBL"

# As regional domains vary so much in purpose, simulation length, and extent, we don't want to assume a minimum duration
## Thus, we ignore start and end dates and simply reduce/output over the whole time frame for all of the examples given.
start_date = None  # "0001-01-01"
end_date = None  # "0101-01-01

save_figs = False
fig_output_dir = None

lc_kwargs = {}
serial = False

## Notebook Specific Arguments
# case_root is automatically populated in a CESM CUPiD workflow
# If case_root is specified, and mom6_input_dir is not, INPUTDIR will be inferred from user_nl_mom in case_root
# If mom6_input_dir is specified, case_root will be ignored
# NOTE: If case_root and mom6_input_dir are both None, the notebook will not run
# If obc_file_str is unspecified, defaults to "forcing_obc_segment_00*.nc"
case_root = None
mom6_input_dir = None  # "/glade/campaign/cgd/oce/projects/CROCODILE/workshops/2025/Diagnostics/Input_Dir/CROCODILE_tutorial_nwa12_MARBL_ocnice"
obc_file_str = None  # Pattern for OBC file names: "forcing_obc_segment_00*.nc"
ocn_variables = []  # ['uo', 'vo', 'thetao', 'so']

In [ ]:
# Parameters for testing (from default examples/regional_ocean/config.yml)
case_name = "CROCODILE_tutorial_nwa12_MARBL"
CESM_output_dir = (
    "/glade/campaign/cgd/oce/projects/CROCODILE/workshops/2025/Diagnostics/CESM_Output/"
)
start_date = ""
end_date = ""
save_figs = True
fig_output_dir = None
lc_kwargs = {"threads_per_worker": 1}
serial = True

case_root = None
mom6_input_dir = "/glade/campaign/cgd/oce/projects/CROCODILE/workshops/2025/Diagnostics/Input_Dir/CROCODILE_tutorial_nwa12_MARBL_ocnice/ocnice"
obc_file_str = "forcing_obc_segment_00*.nc"
ocn_variables = ["uo", "vo", "thetao", "so"]

subset_kwargs = {}

In [ ]:
OUTDIR = f"{CESM_output_dir}/{case_name}/ocn/hist/"
print("Output directory is:", OUTDIR)

In [ ]:
case_output_dir = os.path.join(CESM_output_dir, case_name, "ocn", "hist")

# Xarray time decoding things
time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)

## Static data includes hgrid, vgrid, bathymetry, land/sea mask
static_data = xr.open_mfdataset(
    os.path.join(case_output_dir, f"*static.nc"),
    decode_timedelta=True,
    decode_times=time_coder,
)

## Monthly Domain Data
monthly_data = xr.open_mfdataset(
    os.path.join(case_output_dir, f"*z*.nc"),
    decode_timedelta=True,
    decode_times=time_coder,
)

## Image/Gif Output Directory
if fig_output_dir is None:
    image_output_dir = os.path.join(
        "/glade/derecho/scratch/",
        os.environ["USER"],
        "archive",
        case_name,
        "ocn",
        "cupid_images",
    )
else:
    image_output_dir = os.path.join(fig_output_dir, case_name, "ocn", "cupid_images")
if not os.path.exists(image_output_dir):
    os.makedirs(image_output_dir)
print("Image output directory is:", image_output_dir)

In [ ]:
## Select for only the variables we want to analyze
if len(ocn_variables) > 0:
    print("Selecting only the following monthly variables:", ocn_variables)
    monthly_data = monthly_data[ocn_variables]

## Apply time boundaries
## if they are the right format
if len(start_date.split("-")) == 3 and len(end_date.split("-")) == 3:
    import cftime

    calendar = monthly_data.time.encoding.get("calendar", "standard")

    calendar_map = {
        "gregorian": cftime.DatetimeProlepticGregorian,
        "noleap": cftime.DatetimeNoLeap,
    }

    CFTime = calendar_map.get(calendar, cftime.DatetimeGregorian)
    y, m, d = [int(i) for i in start_date.split("-")]
    start_date_time = CFTime(y, m, d)
    y, m, d = [int(i) for i in end_date.split("-")]
    end_date_time = CFTime(y, m, d)

    print(
        f"Applying time range from start_date: {start_date_time} and end_date: {end_date_time}."
    )

    monthly_data = monthly_data.sel(time=slice(start_date_time, end_date_time))

monthly_time_bounds = [
    monthly_data["time"].isel(time=0).compute().item(),
    monthly_data["time"].isel(time=-1).compute().item(),
]

print(
    f"Monthly Data Time Bounds: {monthly_time_bounds[0]} to {monthly_time_bounds[-1]}"
)

## Accessing and Processing Open Boundary Conditions

#### Regional MOM6 Input Directory

Regional MOM6 configurations use a separate input directory containing the data for the initial state, tides, and boundary conditions. This notebook needs to access the boundary conditions. There are a couple options here:

We instead look for the input directory path in two locations in the `case_root` directory:
- `user_nl_mom`
- `Buildconf/momconf/MOM_input`
  
In both files, the input directory path should have the name `INPUTDIR`. If this path cannot be found, the rest of this notebook will fail to run, but the user can manually set the input directory path below. 

In [ ]:
if mom6_input_dir is None:
    for file in ["user_nl_mom", "BuildConf/momconf/MOM_input"]:
        filepath = Path(os.path.join(case_root, file))

        if filepath.is_file():
            with open(filepath, "r") as f:
                for line in f:
                    if line.strip().startswith("INPUTDIR"):
                        key, value = line.split("=", 1)

                        mom6_input_dir = value.strip()
                        break

        if Path(mom6_input_dir).is_dir():
            print(f"Found INPUTDIR in {file}")
            break

if mom6_input_dir is None:
    print(
        "INPUTDIR not found. Try Setting it manually in the `Regional_Ocean_OBC` notebook."
    )
    raise FileNotFoundError(
        "INPUTDIR not found. Try Setting it manually in the `Regional_Ocean_OBC` notebook."
    )

print(f"INPUTDIR: {mom6_input_dir}")

#### Check for Boundary Conditions, Verify Horizontal Grid

The input directory should contain a few different files:
- `init*`: initial fields
- `ocean_hgrid.nc`, `ocean_topo.nc`, `vgrid*.nc`: grids and bathymetry
- `tu*` and `tz*`: tidal forcing
  
What we care about:
- **`forcing_obc_segment*.nc`**: open boundary conditions to force the model

The numbers at the end of the file each correspond to a different edge of the domain. 
- 001: south
- 002: north
- 003: west
- 004: east

In [ ]:
boundary_conds = glob.glob(os.path.join(mom6_input_dir, obc_file_str))
print(f"Found {len(boundary_conds)} OBC files:", boundary_conds)

if len(boundary_conds) == 0:
    raise FileNotFoundError(f"No boundary condition files found at {mom6_input_dir}")

#### Opening and Checking Open Boundary Conditions

In [ ]:
segment_num_to_dir = {  # could be added to config.yml in the future
    "001": "south",
    "002": "north",
    "003": "west",
    "004": "east",
}
segment_dir_to_num = {v: k for k, v in segment_num_to_dir.items()}

In [ ]:
list_boundaries = []
temp_time_slice = slice(0, 5)

for boundary_file in boundary_conds:

    boundary = xr.open_mfdataset(boundary_file)

    # Temp time slice
    boundary = boundary.isel(time=temp_time_slice)

    code = (boundary_file.split("_")[-1]).split(".")[0]
    bound = segment_num_to_dir[code]

    # Variables on the U/V grid (corners)
    uv_vars = [f"u_segment_{code}", f"v_segment_{code}"]

    # Variables on the T grid (cell center)
    tracer_vars = [
        f"salt_segment_{code}",
        f"temp_segment_{code}",
        f"eta_segment_{code}",
    ]

    # Create separate datasets for each grid type
    boundary_uv = boundary[uv_vars]
    boundary_t = boundary[tracer_vars]

    # Rename coordinates for both datasets
    boundary_uv = boundary_uv.rename(
        {
            f"ny_segment_{code}": "yq",
            f"nx_segment_{code}": "xq",
            f"lon_segment_{code}": "longitude_q",
            f"lat_segment_{code}": "latitude_q",
        }
    )
    boundary_t = boundary_t.rename(
        {
            f"ny_segment_{code}": "yh",
            f"nx_segment_{code}": "xh",
            f"lon_segment_{code}": "longitude_h",
            f"lat_segment_{code}": "latitude_h",
        }
    )

    # Drop variables and handle squeeze for both
    boundary_uv = boundary_uv.drop_vars(["nxp", "nyp"])
    boundary_t = boundary_t.drop_vars(["nxp", "nyp"])

    if bound in ["north", "south"]:
        # Slicing for a C-grid
        boundary_uv = boundary_uv.isel(xq=slice(0, None, 2))
        boundary_t = boundary_t.isel(xh=slice(1, None, 2))

        # Assign new coordinates
        boundary_uv = boundary_uv.assign_coords(
            xq=("xq", boundary_uv["longitude_q"].values)
        )
        boundary_uv = boundary_uv.rename({"yq": "yh"})
        boundary_t = boundary_t.assign_coords(
            xh=("xh", boundary_t["longitude_h"].values)
        )

        # boundary_uv = boundary_uv.squeeze('yq', drop = True)
        # boundary_t = boundary_t.squeeze('yh', drop = True)
    else:
        # Slicing for a C-grid
        boundary_uv = boundary_uv.isel(yq=slice(0, None, 2))
        boundary_t = boundary_t.isel(yh=slice(1, None, 2))

        # Assign new coordinates
        boundary_uv = boundary_uv.assign_coords(
            yq=("yq", boundary_uv["latitude_q"].values)
        )
        boundary_uv = boundary_uv.rename({"xq": "xh"})
        boundary_t = boundary_t.assign_coords(
            yh=("yh", boundary_t["latitude_h"].values)
        )

        # boundary_uv = boundary_uv.squeeze('xq', drop = True)
        # boundary_t = boundary_t.squeeze('xh', drop = True)

    # Combine the processed datasets
    boundary = xr.merge([boundary_uv, boundary_t])

    # Remove longitude and latitude variables from the datasets
    for coord in ["longitude_q", "latitude_q"]:
        if coord in boundary:
            boundary = boundary.drop_vars(coord)
    for coord in ["longitude_h", "latitude_h"]:
        if coord in boundary:
            boundary = boundary.drop_vars(coord)

    # Dumb check to make sure OBCs are on the same grid as model output
    model_xh = static_data["xh"]
    model_yh = static_data["yh"]
    model_xq = static_data["xq"]
    model_yq = static_data["yq"]

    z_map = [
        f"nz_segment_{code}_u",
        f"nz_segment_{code}_v",
        f"nz_segment_{code}_temp",
        f"nz_segment_{code}_salt",
    ]
    new_boundary = {}
    for var in boundary.data_vars:
        da = boundary[var]
        da.load()

        zdim = [str(d) for d in da.dims if str(d) in z_map]
        if len(zdim) > 0:
            da = da.rename({zdim[0]: "z_l"})
        new_boundary[var.split("_")[0]] = da

    new_boundary = xr.Dataset(new_boundary).expand_dims(boundary=[bound])
    list_boundaries.append(new_boundary)

## Time to plot!
## 🚧 IN PROGRESS 🚧 

In [ ]:
print("END OF NOTEBOOK FOR NOW.")

In [ ]:
# # Alternative approach: Use utils.plotLatLonField and add boundaries manually
# def plot_surface_with_boundaries_simple(var_name, monthly_data, static_data, boundaries,
#                                        save=False, save_path=None):
#     """
#     Simple approach to plot surface fields with boundary overlays
#     """
#     if var_name not in monthly_data:
#         print(f"Variable '{var_name}' not found. Skipping.")
#         return

#     # Select the first time step
#     field = monthly_data[var_name].isel(time=0)

#     # Get coordinates
#     coords = utils.chooseGeoCoords(field.dims)
#     areacello = utils.chooseAreacello(field.dims)

#     # Use the existing plotLatLonField function
#     fig, ax = utils.plotLatLonField(
#         field,
#         latitude=static_data[coords["latitude"]],
#         longitude=static_data[coords["longitude"]],
#         stats=True,
#         area_weights=static_data[areacello],
#         save=False,  # We'll handle saving ourselves
#         save_path=None,
#         return_axis=True  # Assuming this parameter exists, otherwise we'll work around it
#     )

#     # If the function doesn't return axis, create our own plot
#     if ax is None:
#         fig = plt.figure(figsize=(12, 8), dpi=150)
#         ax = plt.axes(projection=ccrs.Mercator())

#         # Recreate the plot manually
#         latitude = static_data[coords["latitude"]]
#         longitude = static_data[coords["longitude"]]

#         field_with_coords = field.assign_coords({
#             "lon": longitude,
#             "lat": latitude
#         })

#         cmap = utils.chooseColorMap(var_name)
#         levels = utils.chooseColorLevels(
#             field.min().compute().item(),
#             field.max().compute().item()
#         )

#         p = field_with_coords.plot(
#             x="lon", y="lat",
#             cmap=cmap,
#             levels=levels,
#             transform=ccrs.PlateCarree(),
#             ax=ax,
#             add_colorbar=True
#         )

#         ax.coastlines(resolution="50m", color="black", linewidth=0.5)

#     # Add boundary overlays
#     boundary_colors = ['red', 'blue', 'green', 'orange']

#     for i, boundary_ds in enumerate(boundaries):
#         color = boundary_colors[i % len(boundary_colors)]
#         boundary_name = boundary_ds.boundary.values[0]

#         # Plot boundary outline - simplified approach
#         # We'll plot the boundary as points or lines based on available data

#         # Get a representative variable from the boundary
#         boundary_vars = list(boundary_ds.data_vars)
#         if len(boundary_vars) > 0:
#             rep_var = boundary_ds[boundary_vars[0]]

#             # Create boundary markers based on the boundary location
#             if boundary_name in ['north', 'south']:
#                 # For north/south boundaries, plot horizontal lines
#                 if 'xh' in rep_var.dims:
#                     x_coords = static_data['geolon'].isel(yh=0 if boundary_name == 'south' else -1)
#                     y_coords = static_data['geolat'].isel(yh=0 if boundary_name == 'south' else -1)
#                     ax.plot(x_coords, y_coords, color=color, linewidth=4,
#                            label=f'{boundary_name} boundary', transform=ccrs.PlateCarree(),
#                            marker='o', markersize=2)

#             elif boundary_name in ['east', 'west']:
#                 # For east/west boundaries, plot vertical lines
#                 if 'yh' in rep_var.dims:
#                     x_coords = static_data['geolon'].isel(xh=0 if boundary_name == 'west' else -1)
#                     y_coords = static_data['geolat'].isel(xh=0 if boundary_name == 'west' else -1)
#                     ax.plot(x_coords, y_coords, color=color, linewidth=4,
#                            label=f'{boundary_name} boundary', transform=ccrs.PlateCarree(),
#                            marker='o', markersize=2)

#     # Add legend
#     ax.legend(loc='upper left', bbox_to_anchor=(0.02, 0.98))
#     ax.set_title(f"{field.attrs.get('long_name', var_name)} with Open Boundary Conditions",
#                 fontsize=12, pad=15)

#     # Save if requested
#     if save and save_path:
#         filename = f"{var_name}_with_obc.png"
#         filepath = os.path.join(save_path, filename)
#         plt.savefig(filepath, dpi=300, bbox_inches='tight', facecolor='white')
#         print(f"Saved: {filepath}")

#     return fig, ax

# for var in ocn_variables:
#     if var in monthly_data:
#         print(f"\nPlotting {var}...")
#         fig, ax = plot_surface_with_boundaries_simple(
#             var, monthly_data, static_data, list_boundaries,
#             save=save_figs, save_path=image_output_dir
#         )
#         plt.show()
#     else:
#         print(f"Variable {var} not available in surface data")